In [1]:
from sklearn.datasets import load_wine
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shap
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import xgboost as xgb
from scipy.spatial import ConvexHull, Delaunay
import pickle
import os
import time
# import torch
# import torch.nn as nn
from tqdm.auto import tqdm
import numpy as np

In [3]:
hdbscan_csv = pd.read_csv('/Users/swarchol/Research/seal/data/updated_renamed_with_hdbscan.csv')

In [4]:
# save to parquet
hdbscan_csv.to_parquet('/Users/swarchol/Research/seal/data/updated_renamed_with_hdbscan.parquet')


In [5]:
# read contours as pkl
# /Users/swarchol/Research/seal/data/greg.contour.pkl
contours = pickle.load(open('/Users/swarchol/Research/seal/data/greg.contour.pkl', 'rb'))

In [6]:
# dump to json
with open('/Users/swarchol/Research/seal/data/greg.contour.json', 'w') as f:
    json.dump(contours, f)

# read json
contours = json.load(open('/Users/swarchol/Research/seal/data/greg.contour.json', 'r'))


In [7]:
# read shap 
shap = np.load('/Users/swarchol/Research/seal/data/greg.shap.npy')

In [12]:

def get_potential_features(df):
    # Check which of the potential features are in the csv_df
    all_features = [
        "ELANE",
        "CD57",
        "CD45",
        "CD11B",
        "SMA",
        "CD16",
        "ECAD",
        "FOXP3",
        "NCAM",
        "anti_CD3",
        "anti_CD45RO",
        "Keratin_570",
        "aSMA_660",
        "CD4_488",
        "CD45_PE",
        "PD1_647",
        "CD20_488",
        "CD68_555",
        "CD8a_660",
        "CD163_488",
        "FOXP3_570",
        "PDL1_647",
        "Ecad_488",
        "Vimentin_555",
        "CDX2_647",
        "LaminABC_488",
        "Desmin_555",
        "CD31_647",
        "PCNA_488",
        "CollagenIV_647",
        "CD3",
        "CD45RO",
        "Pan-cytokeratin",
        "Aortic smooth muscle actin",
        "CD4",
        "CD45",
        "PD-1",
        "CD20",
        "CD68",
        "CD8a",
        "CD163",
        "FOXP3",
        "PD-L1",
        "E-cadherin",
        "Vimentin",
        "CDX-2",
        "Lamin-A/B/C",
        "Desmin",
        "CD31",
        "PCNA",
        "Collagen",
        "DNA",
        "DNA (2)",
        "DNA (3)",
        "CD3",
        "CD45RO",
        "DNA (4)",
        "Pan-cytokeratin",
        "Aortic smooth muscle actin",
        "DNA (5)",
        "CD4",
        "CD45",
        "PD-1",
        "DNA (6)",
        "CD20",
        "CD68",
        "CD8a",
        "DNA (7)",
        "CD163",
        "FOXP3",
        "PD-L1",
        "DNA (8)",
        "E-cadherin",
        "Vimentin",
        "CDX-2",
        "DNA (9)",
        "Lamin-A/B/C",
        "Desmin",
        "CD31",
        "DNA (10)",
        "PCNA",
        "Collagen",
        "u",
        "g",
        "r",
        "i",
        "z",
        "JPG_R",
        "JPG_G",
        "JPG_B" "DNA1",
        "AF1",
        "CD31",
        "Ki67",
        "CD68",
        "CD163",
        "CD20",
        "CD4",
        "CD8a",
        "CD11c",
        "PDL1",
        "CD3e",
        "ECAD",
        "PD1",
        "FOXP3",
        "CD45",
        "SOX10",
        "pH3",
        "u",
        "g",
        "r",
        "i",
        "z",
        # "petroRad_r",  # Petrosian radius in the r-band – an estimate of the size of an object (usually a galaxy).
        #  Probability the object is a point source (i.e., star-like) based on shape. High values suggest stars, while extended objects (like galaxies) have lower values.
        "cz",
        "extinction_r",
        "airmass_r",
        "mCr4_r",
        'CD11B', 'CD16', 'CD45', 'CD57', 'DNA_6', 'DNA_7', 'DNA_8', 'ECAD', 'ELANE', 'FOXP3', 'NCAM', 'SMA'
    ]

    all_features = list(set(all_features))
    potential_features = [feature for feature in all_features if feature in df.columns]
    return sorted(potential_features)


In [10]:
test = pd.read_parquet('https://seal-vis.s3.us-east-1.amazonaws.com/exemplar-001/shap.parquet')

In [15]:
feature_columns = get_potential_features(hdbscan_csv)
feature_columns, len(feature_columns)

(['Aortic smooth muscle actin',
  'CD163',
  'CD20',
  'CD3',
  'CD31',
  'CD4',
  'CD45',
  'CD45RO',
  'CD68',
  'CD8a',
  'CDX-2',
  'Collagen',
  'DNA',
  'DNA (10)',
  'DNA (2)',
  'DNA (3)',
  'DNA (4)',
  'DNA (5)',
  'DNA (6)',
  'DNA (7)',
  'DNA (8)',
  'DNA (9)',
  'Desmin',
  'E-cadherin',
  'FOXP3',
  'Lamin-A/B/C',
  'PCNA',
  'PD-1',
  'PD-L1',
  'Pan-cytokeratin',
  'Vimentin'],
 31)

In [17]:
test

,CD11B,CD16,CD45,CD57,DNA_6,DNA_7,DNA_8,ECAD,ELANE,FOXP3,NCAM,SMA
0,49.246490,27.609840,245.884445,135.754684,46.078102,315.915283,100.273346,110.053116,49.234573,41.485001,11.474500,1205.727295
1,1.861968,50.019844,157.713913,123.427170,38.683456,97.170464,12.428247,127.096695,5.289040,35.351128,4.598541,609.455444
2,5.953567,81.587921,135.455963,123.446968,177.626801,318.460480,278.353851,51.848412,63.182808,28.096024,12.778870,833.832336
3,29.503378,52.394604,226.204559,119.526260,24.948776,77.673882,25.696209,78.605133,22.764359,40.152523,7.794022,678.739014
4,6.299270,7.443626,401.430603,181.437698,186.350677,59.693291,111.020668,39.586758,2.968853,22.789564,8.507957,900.096313
...,...,...,...,...,...,...,...,...,...,...,...,...
11165,21.195604,71.242996,290.291992,105.904648,111.876854,139.243256,21.441109,112.024506,28.372923,74.308617,4.127245,429.708923
11166,18.768259,91.498955,508.812622,98.766464,37.437248,63.982765,21.297955,110.135040,37.451992,70.889610,28.299652,326.373077
11167,24.648407,68.916351,371.123138,106.268982,53.525288,61.175621,30.619579,102.593948,17.089479,64.009644,4.075585,566.222168
11168,34.575905,79.509552,357.701965,114.702820,55.475220,65.986603,39.358414,132.345367,24.791027,72.548904,19.472546,356.762329


In [22]:
shap = np.abs(shap)
shap_sum = np.sum(shap, axis=2)
shap_sum.shape


(933253, 31)

In [25]:
shap_df = pd.DataFrame(shap_sum)
shap_df.columns = feature_columns

In [27]:
shap_df.to_parquet('/Users/swarchol/Research/seal/data/greg.shap_sum.parquet')